In [6]:
from elasticsearch import Elasticsearch
es = Elasticsearch("orikami:wbwZo0Pp755kRmpudmWk@mozart-es.orikami.eu:9200")


In [7]:
def getQuery(artists, title):
        combined = ''
        if artists == None :
            combined = {
                "multi_match": {
                    "query" : title,
                    "type" : "cross_fields",
                    "fields": [
                        "title",
                        "artists"
                    ],
                    "fuzziness" : "AUTO",
                    "minimum_should_match": "2<70%"
            }
        }

        else :
         combined = {
                "bool": {
                    "must": [
                        {
                            "match": {
                                "artists": {
                                    "query"    : artists,
                                    "fuzziness": "AUTO"
                                }
                            }
                        },
                        {
                            "match": {
                                "title": {
                                    "query"    : title,
                                    "fuzziness": "AUTO"
                                }
                            }
                        }
                    ]
                }
            }

        query = {
            "query": {
                "function_score": {
                    "query": {
                        "boosting": {
                            "positive": combined,
                            "negative": {
                                "term": {
                                    "releases": "live"
                                }
                            },
                            "negative_boost": 0.7
                        }
                    },
                    "functions": [
                        {
                            "field_value_factor": {
                                "field": "recording_score"
                            }
                        },
                        {
                            "filter": {
                                "term": {
                                    "release_statusses": "official"
                                }
                            },
                            "boost_factor": 1.2
                        },
                        {
                            "filter": {
                                "term": {
                                    "release_types": "album"
                                }
                            },
                            "boost_factor": 1.05
                        },
                        {
                            "field_value_factor": {
                                "field": "no_releases",
                                "modifier": "log1p"
                            },
                            "weight": "0.2"
                        }
                    ],
                    "score_mode": "sum"
                }
            },
            "filter": {
                "bool": {
                    "should": [
                        {
                            "term": {
                                "release_statusses": "official"
                            }
                        },
                        {
                            "missing": {
                                "field": "release_statusses"
                            }
                        }
                    ]
                }
            },
            "fields": [
                "mb_id",
                "artists",
                "title"
            ],
            "size": 3
        };

        return query



query = getQuery(artists="Mr probz", title="waves")

results = es.search(index="recordings", body=query)
print(results)

{'_shards': {'total': 5, 'successful': 5, 'failed': 0}, 'hits': {'total': 88, 'max_score': 14.336902, 'hits': [{'fields': {'title': ['Waves'], 'artists': ['Mr. Probz'], 'mb_id': ['ca9abf2b-a822-4664-8967-d8da71d3802d']}, '_score': 14.336902, '_type': 'recording', '_index': 'recordings', '_id': 'ca9abf2b-a822-4664-8967-d8da71d3802d'}, {'fields': {'title': ['Waves (Robin Schulz remix)'], 'artists': ['Mr. Probz'], 'mb_id': ['201d06ec-3d74-469d-8645-f2c89b87b58a']}, '_score': 12.351844, '_type': 'recording', '_index': 'recordings', '_id': '201d06ec-3d74-469d-8645-f2c89b87b58a'}, {'fields': {'title': ['Waves (Robin Schulz radio edit)'], 'artists': ['Mr. Probz'], 'mb_id': ['913b4327-2282-48fe-8e42-c90cba6fb7e5']}, '_score': 12.078592, '_type': 'recording', '_index': 'recordings', '_id': '913b4327-2282-48fe-8e42-c90cba6fb7e5'}]}, 'timed_out': False, 'took': 49}
